In [1]:
from pyalgotrade.tools import yahoofinance

In [2]:
# 耐心等待csv的出现...
yahoofinance.download_daily_bars('tcehy', 2016, 'tcehy-2016.csv')

In [3]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed

In [4]:
# 简单输出close
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.__instrument = instrument

    def onBars(self, bars):
        bar = bars[self.__instrument]
        self.info(bar.getClose())

In [12]:
from pyalgotrade.technical import ma
from pyalgotrade.technical import rsi

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.__rsi = rsi.RSI(feed[instrument].getCloseDataSeries(), 14)
        self.__sma = ma.SMA(self.__rsi, 15)
        self.__instrument = instrument

    def onBars(self, bars):
        bar = bars[self.__instrument]
        self.info("%s %s %s" % (bar.getClose(), self.__rsi[-1], self.__sma[-1]))

In [13]:
# Load the yahoo feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("tcehy", "tcehy-2016.csv")

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "tcehy")
myStrategy.run()

2016-01-04 00:00:00 strategy [INFO] 19.129999 None None
2016-01-05 00:00:00 strategy [INFO] 19.08 None None
2016-01-06 00:00:00 strategy [INFO] 18.66 None None
2016-01-07 00:00:00 strategy [INFO] 17.91 None None
2016-01-08 00:00:00 strategy [INFO] 17.969999 None None
2016-01-11 00:00:00 strategy [INFO] 17.709999 None None
2016-01-12 00:00:00 strategy [INFO] 17.75 None None
2016-01-13 00:00:00 strategy [INFO] 17.24 None None
2016-01-14 00:00:00 strategy [INFO] 17.59 None None
2016-01-15 00:00:00 strategy [INFO] 17.09 None None
2016-01-19 00:00:00 strategy [INFO] 17.709999 None None
2016-01-20 00:00:00 strategy [INFO] 17.200001 None None
2016-01-21 00:00:00 strategy [INFO] 17.32 None None
2016-01-22 00:00:00 strategy [INFO] 18.120001 None None
2016-01-25 00:00:00 strategy [INFO] 18.299999 41.9729113806 None
2016-01-26 00:00:00 strategy [INFO] 18.540001 44.7357395039 None
2016-01-27 00:00:00 strategy [INFO] 18.02 40.2627254955 None
2016-01-28 00:00:00 strategy [INFO] 18.280001 43.31463804

In [17]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        strategy.BacktestingStrategy.__init__(self, feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 10, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()

In [19]:
def run_strategy(smaPeriod):
    # Load the yahoo feed from the CSV file
    feed = yahoofeed.Feed()
    feed.addBarsFromCSV("tcehy", "tcehy-2016.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "tcehy", smaPeriod)
    myStrategy.run()
    print "Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity()

run_strategy(15)

2016-01-26 00:00:00 strategy [INFO] BUY at $18.09
2016-02-08 00:00:00 strategy [INFO] SELL at $17.83
2016-02-18 00:00:00 strategy [INFO] BUY at $18.15
2016-04-06 00:00:00 strategy [INFO] SELL at $20.30
2016-04-07 00:00:00 strategy [INFO] BUY at $20.73
2016-04-22 00:00:00 strategy [INFO] SELL at $20.64
2016-05-17 00:00:00 strategy [INFO] BUY at $20.66
2016-06-14 00:00:00 strategy [INFO] SELL at $21.62
2016-06-16 00:00:00 strategy [INFO] BUY at $21.80
2016-06-17 00:00:00 strategy [INFO] SELL at $21.81
2016-06-22 00:00:00 strategy [INFO] BUY at $22.87
2016-06-27 00:00:00 strategy [INFO] SELL at $21.74
2016-06-30 00:00:00 strategy [INFO] BUY at $22.65


Final portfolio value: $1019.75


In [14]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade.technical import cross


class SMACrossOver(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.__instrument = instrument
        self.__position = None
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__prices = feed[instrument].getPriceDataSeries()
        self.__sma = ma.SMA(self.__prices, smaPeriod)

    def getSMA(self):
        return self.__sma

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if cross.cross_above(self.__prices, self.__sma) > 0:
                shares = int(self.getBroker().getCash() * 0.9 / bars[self.__instrument].getPrice())
                # Enter a buy market order. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, shares, True)
        # Check if we have to exit the position.
        elif not self.__position.exitActive() and cross.cross_below(self.__prices, self.__sma) > 0:
            self.__position.exitMarket()

In [ ]:
from pyalgotrade import plotter
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.stratanalyzer import returns

# Load the yahoo feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("tcehy", "tcehy-2016.csv")

# Evaluate the strategy with the feed's bars.
myStrategy = SMACrossOver(feed, "tcehy", 20)

# Attach a returns analyzers to the strategy.
returnsAnalyzer = returns.Returns()
myStrategy.attachAnalyzer(returnsAnalyzer)

# Attach the plotter to the strategy.
plt = plotter.StrategyPlotter(myStrategy)
# Include the SMA in the instrument's subplot to get it displayed along with the closing prices.
plt.getInstrumentSubplot("tcehy").addDataSeries("SMA", myStrategy.getSMA())
# Plot the simple returns on each bar.
plt.getOrCreateSubplot("returns").addDataSeries("Simple returns", returnsAnalyzer.getReturns())

# Run the strategy.
myStrategy.run()
myStrategy.info("Final portfolio value: $%.2f" % myStrategy.getResult())

# Plot the strategy.
plt.plot()

2016-07-01 00:00:00 strategy [INFO] Final portfolio value: $1209441.71
